# Fact or Fake: News Analysis

## Data Mining 334
## Alex Laughlin, Xandre Clementsmith

## Imports

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer

#pull in data

# switch politifact to gossipcop to try other dataset
df_p_real = pd.read_csv('politifact_real.csv')
df_p_fake = pd.read_csv('politifact_fake.csv')

df_p_real['true/false'] = True
df_p_real.head(5)

,id,news_url,title,tweet_ids,true/false
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,True
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,True
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN,True
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN,True
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...,True


## Dataframe Creation and Manipulation

In [16]:
df_p_fake['true/false'] = False
df_p_fake.head(5)

,id,news_url,title,tweet_ids,true/false
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,False
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,False
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,False
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,False
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,False


In [17]:
#Join tables together
frames = [df_p_real, df_p_fake]
df = pd.concat(frames)
df.head(5)
df.shape

(1056, 5)

## Tokenization

In [18]:
stemmer = SnowballStemmer("english")

tokenizer = nltk.RegexpTokenizer(r"\w+")

#remove special characters
df['title'] = df.apply(lambda row: re.sub('[^A-Za-z0-9 ]+', '', str(row['title'])), axis =1)

#tokenize words from title
df['tokenized_sents'] = df.apply(lambda row: tokenizer.tokenize(row['title']), axis=1)

#remove stop words from tokenized titles
df['tokens_without_stopwords'] = df['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop_words])

#stem tokenized words without stopwords
df['tokens_stemmed']=df['tokens_without_stopwords'].apply(lambda x : [stemmer.stem(y) for y in x])

#lemmatize tokenized words without stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
df['tokens_lemmatized'] = df['tokens_without_stopwords'].apply(lambda x : [lemmatizer.lemmatize(y) for y in x])
df.head(10)

,id,news_url,title,tweet_ids,true/false,tokenized_sents,tokens_without_stopwords,tokens_stemmed,tokens_lemmatized
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,True,"[National, Federation, of, Independent, Business]","[National, Federation, Independent, Business]","[nation, feder, independ, busi]","[National, Federation, Independent, Business]"
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,True,"[comments, in, Fayetteville, NC]","[comments, Fayetteville, NC]","[comment, fayettevill, nc]","[comment, Fayetteville, NC]"
2,politifact333,https://web.archive.org/web/20080204072132/htt...,Romney makes pitch hoping to close deal Elect...,NaN,True,"[Romney, makes, pitch, hoping, to, close, deal...","[Romney, makes, pitch, hoping, close, deal, El...","[romney, make, pitch, hope, close, deal, elect...","[Romney, make, pitch, hoping, close, deal, Ele..."
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN,True,"[Democratic, Leaders, Say, House, Democrats, A...","[Democratic, Leaders, Say, House, Democrats, A...","[democrat, leader, say, hous, democrat, are, u...","[Democratic, Leaders, Say, House, Democrats, A..."
4,politifact779,https://web.archive.org/web/20070820164107/htt...,Budget of the United States Government FY 2008,89804710374154240\t91270460595109888\t96039619...,True,"[Budget, of, the, United, States, Government, ...","[Budget, United, States, Government, FY, 2008]","[budget, unit, state, govern, fy, 2008]","[Budget, United, States, Government, FY, 2008]"
5,politifact14064,http://www.politifact.com/truth-o-meter/statem...,Donald Trump exaggerates when he says China ha...,690248006399049728\t690254026663821312\t690276...,True,"[Donald, Trump, exaggerates, when, he, says, C...","[Donald, Trump, exaggerates, says, China, tota...","[donald, trump, exagger, say, china, total, co...","[Donald, Trump, exaggerates, say, China, total..."
6,politifact14474,https://www.law.cornell.edu/constitution/amend...,25th Amendment,1262604762\t10969740933\t11182364398\t17507543...,True,"[25th, Amendment]","[25th, Amendment]","[25th, amend]","[25th, Amendment]"
7,politifact5276,http://americaneedsmitt.com/blog/2011/11/10/mi...,,NaN,True,[],[],[],[]
8,politifact1313,https://web.archive.org/web/20090913221204/htt...,Briefing by White House Press Secretary Robert...,13511762265\t13512918230\t13513835900\t1351424...,True,"[Briefing, by, White, House, Press, Secretary,...","[Briefing, White, House, Press, Secretary, Rob...","[brief, white, hous, press, secretari, robert,...","[Briefing, White, House, Press, Secretary, Rob..."
9,politifact937,https://web.archive.org/web/20080623122709/htt...,A Solar Grand Plan Scientific American,140962137332920320\t141057766704947200\t141166...,True,"[A, Solar, Grand, Plan, Scientific, American]","[A, Solar, Grand, Plan, Scientific, American]","[a, solar, grand, plan, scientif, american]","[A, Solar, Grand, Plan, Scientific, American]"


## Dictionary and IDF

In [19]:
# MODULAR COMPONENT:
# select token_type for machine learning algorithms
# tokens_stemmed performs best
token_type = 'tokens_stemmed'

In [20]:
#create corpus from lemmatized tokens as numpy array
corpus = df[token_type].to_numpy()

#create dictionary
DF = {}

#write unique words to dictionary
for i in range(len(corpus)):
    tokens = corpus[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            
#get word frequency
for i in DF:
    DF[i] = len(DF[i])
# print(DF)

In [21]:
all_words = [x for x in DF]
# print(all_words)

In [22]:
#create column of strings from lemmatized tokens
df['tfidfprep']=[" ".join(x) for x in df[token_type].values]
labels=df[["true/false"]]

#create train and test sets
labels=df[["true/false"]]
x_train, x_test, y_train, y_test= train_test_split(df['tfidfprep'], labels, test_size=0.2, stratify=df['true/false'], random_state=7)

#initialize tfidf vectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

#create tfidf of train and test sets
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

df.head()

,id,news_url,title,tweet_ids,true/false,tokenized_sents,tokens_without_stopwords,tokens_stemmed,tokens_lemmatized,tfidfprep
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,True,"[National, Federation, of, Independent, Business]","[National, Federation, Independent, Business]","[nation, feder, independ, busi]","[National, Federation, Independent, Business]",nation feder independ busi
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,True,"[comments, in, Fayetteville, NC]","[comments, Fayetteville, NC]","[comment, fayettevill, nc]","[comment, Fayetteville, NC]",comment fayettevill nc
2,politifact333,https://web.archive.org/web/20080204072132/htt...,Romney makes pitch hoping to close deal Elect...,NaN,True,"[Romney, makes, pitch, hoping, to, close, deal...","[Romney, makes, pitch, hoping, close, deal, El...","[romney, make, pitch, hope, close, deal, elect...","[Romney, make, pitch, hoping, close, deal, Ele...",romney make pitch hope close deal elect the ro...
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN,True,"[Democratic, Leaders, Say, House, Democrats, A...","[Democratic, Leaders, Say, House, Democrats, A...","[democrat, leader, say, hous, democrat, are, u...","[Democratic, Leaders, Say, House, Democrats, A...",democrat leader say hous democrat are unit aga...
4,politifact779,https://web.archive.org/web/20070820164107/htt...,Budget of the United States Government FY 2008,89804710374154240\t91270460595109888\t96039619...,True,"[Budget, of, the, United, States, Government, ...","[Budget, United, States, Government, FY, 2008]","[budget, unit, state, govern, fy, 2008]","[Budget, United, States, Government, FY, 2008]",budget unit state govern fy 2008


## Machine Learning

### Logistic Regression (Baseline)

In [23]:
lr = LogisticRegression(random_state=0)

lr.fit(tfidf_train, y_train.values.ravel())

lr_y_pred = lr.predict(tfidf_test)

lr_score = accuracy_score(y_test, lr_y_pred)
print(f'Accuracy: {round(lr_score*100,2)}%')
print(confusion_matrix(y_test,lr_y_pred))

Accuracy: 81.6%
[[ 57  30]
 [  9 116]]


### Decision Tree

In [24]:
dt = DecisionTreeClassifier(random_state=0)

dt.fit(tfidf_train, y_train.values.ravel())

dt_y_pred = dt.predict(tfidf_test)
dt_features = dt.feature_importances_

dt_score = accuracy_score(y_test, dt_y_pred)
print(f'Accuracy: {round(dt_score*100,2)}%')
print(confusion_matrix(y_test,dt_y_pred))

Accuracy: 78.77%
[[76 11]
 [34 91]]


### RandomForest

In [25]:
rf = RandomForestClassifier(random_state=0)

rf.fit(tfidf_train, y_train.values.ravel())

rf_y_pred = rf.predict(tfidf_test)
rf_features = rf.feature_importances_

rf_score = accuracy_score(y_test, rf_y_pred)
print(f'Accuracy: {round(rf_score*100,2)}%')
print(confusion_matrix(y_test,rf_y_pred))

Accuracy: 84.43%
[[ 69  18]
 [ 15 110]]


### Passive Aggresive Classifier

In [26]:
#instantiate passive aggresive classifier
pac=PassiveAggressiveClassifier(max_iter=100)

#fit classifer to tfidf of training set
pac.fit(tfidf_train,y_train.values.ravel())

#create predictions on test set
pac_y_pred = pac.predict(tfidf_test)

#print score
pac_score = accuracy_score(y_test, pac_y_pred)
print(f'Accuracy: {round(pac_score*100,2)}%')
print(confusion_matrix(y_test,pac_y_pred))

Accuracy: 85.85%
[[ 73  14]
 [ 16 109]]
